<a href="https://colab.research.google.com/github/edivandocouto/Spatial-Data-Analysis/blob/main/ct_mapbiomas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#Instalando as bibliotecas
!pip install -q geemap==0.16.0
!pip install earthengine-api
!pip install geopandas

     |████████████████████████████████| 2.0 MB 5.2 MB/s 
     |████████████████████████████████| 8.8 MB 51.6 MB/s 
     |████████████████████████████████| 46 kB 4.2 MB/s 
     |████████████████████████████████| 98 kB 7.2 MB/s 
     |████████████████████████████████| 1.3 MB 62.0 MB/s 
     |████████████████████████████████| 130 kB 45.3 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 3.0 MB 46.7 MB/s 
     |████████████████████████████████| 3.4 MB 40.8 MB/s 
     |████████████████████████████████| 198 kB 72.2 MB/s 
     |████████████████████████████████| 99 kB 6.8 MB/s 
     |████████████████████████████████| 1.2 MB 60.5 MB/s 
     |████████████████████████████████| 128 kB 68.5 MB/s 
     |████████████████████████████████| 1.6 MB 58.7 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 9.8 MB 38.2 MB/s 
     |████████████████████████████████| 346 kB 60.7 MB/s 
     |███████████████████

In [9]:
#Importando as bibliotecas
import ee
import geopandas as gpd
import json
import geemap

In [10]:
##Montando o acesso ao google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
##Autenticação do GEE e Inicialização
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=xVc32-rXcG68Mj5y7HKomdN-ZEUySWaY0Yu5Ay1HL_0&tc=4LEnlFrRw6E70dX3EgvrEHNZQ5t7HONAcwWd_acMCeg&cc=hViasAsV7hGT6sBZVjO7MeVdkkMPZlQgKOYDTyPDoXQ

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJrjbbY0St4WbvM77kN-BAN9VuQ0qccYcpiNAO3i7OQ-92AdVZpFUmw

Successfully saved authorization token.


In [ ]:
#Autenticar no Google Cloud
#from google.colab import auth
#auth.authenticate_user()

In [68]:
# Coleção 3 Mapbiomas bacia amazonica  
MapBiomasAmazon = (ee.Image('projects/mapbiomas-raisg/public/collection3/mapbiomas_raisg_panamazonia_collection3_integration_v2'))
print(MapBiomasAmazon.getInfo())

{'type': 'Image', 'bands': [{'id': 'classification_1985', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [141216, 117285], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -81.23725625725228, 0, -0.00026949458523585647, 10.372038101972407]}, {'id': 'classification_1986', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [141216, 117285], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -81.23725625725228, 0, -0.00026949458523585647, 10.372038101972407]}, {'id': 'classification_1987', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [141216, 117285], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -81.23725625725228, 0, -0.00026949458523585647, 10.372038101972407]}, {'id': 'classification_1988', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [141216, 117285], 'crs': 

In [69]:
bandlist = [f'classification_{year}' for year in range(1985,2021)]
bandlist

['classification_1985',
 'classification_1986',
 'classification_1987',
 'classification_1988',
 'classification_1989',
 'classification_1990',
 'classification_1991',
 'classification_1992',
 'classification_1993',
 'classification_1994',
 'classification_1995',
 'classification_1996',
 'classification_1997',
 'classification_1998',
 'classification_1999',
 'classification_2000',
 'classification_2001',
 'classification_2002',
 'classification_2003',
 'classification_2004',
 'classification_2005',
 'classification_2006',
 'classification_2007',
 'classification_2008',
 'classification_2009',
 'classification_2010',
 'classification_2011',
 'classification_2012',
 'classification_2013',
 'classification_2014',
 'classification_2015',
 'classification_2016',
 'classification_2017',
 'classification_2018',
 'classification_2019',
 'classification_2020']

In [70]:
MapBiomasAmazon = (ee.Image('projects/mapbiomas-raisg/public/collection3/mapbiomas_raisg_panamazonia_collection3_integration_v2'))
print('Number of images:',MapBiomasAmazon.bandNames().size().getInfo())
amazonCollection = ee.ImageCollection(MapBiomasAmazon.bandNames().map(lambda b: MapBiomasAmazon.select(bandlist)))
print('Collection size:',amazonCollection.size().getInfo())

Number of images: 36
Collection size: 36


In [92]:
asset = 'projects/mapbiomas-raisg/public/collection3/mapbiomas_raisg_panamazonia_collection3_integration_v2'
amazonBiomas = ee.Image(asset)

fromList = [1, 3, 4, 5, 6, 11, 10, 12, 13, 29, 14, 22, 25, 30, 26, 33, 34, 27, 24]
toList = [1, 1, 2, 3, 4, 4, 5, 5, 5, 5, 6, 7, 7, 7, 8, 8, 9, 10, 11]

names = amazonBiomas.bandNames()
#names = names.filter(ee.Filter.inList('item',bandlist))

def reclass(band):
  return amazonBiomas.remap(fromList,toList,0,band).rename([band])

amazonBiomas_reclass = names.map(reclass)

In [ ]:
# valor do pixels - valor do pixel no mapbiomas - código da cor - nome da formação
# 1 - 1-3 '129912',Forest
# 2 – 4 '32CD32',Savanna Formation
# 3- 5 '687537', Mangrove
# 4- 6, 11 ‘70A5AE’ Flooded Forest and Wetland
# 5- 10, 12, 13, 29 ‘BBFCAC’ Other Non Forest Natural Formation
# 6- 14, ‘FFFFB2’ Farming 
# 7- 22,25, 30 ‘EA9999’ Non vegetated area  
# 8- 26,33, ‘0000FF’ Water 
# 9- 34 ‘b5bbc3’ Glacier
# 10- 27 ‘D5D5E5’ Non Observed
# 11 – 24 ‘AA0000’ Urban Infrastructure

In [93]:
amazonBiomas_reclass.getInfo()

[{'type': 'Image',
  'bands': [{'id': 'classification_1985',
    'data_type': {'type': 'PixelType',
     'precision': 'int',
     'min': 0,
     'max': 11},
    'dimensions': [141216, 117285],
    'crs': 'EPSG:4326',
    'crs_transform': [0.00026949458523585647,
     0,
     -81.23725625725228,
     0,
     -0.00026949458523585647,
     10.372038101972407]}],
  'version': 1632375433141026,
  'id': 'projects/mapbiomas-raisg/public/collection3/mapbiomas_raisg_panamazonia_collection3_integration_v2',
  'properties': {'system:footprint': {'type': 'LinearRing',
    'coordinates': [[-63.69537156255461, -21.235769568181407],
     [-60.127558032033875, -21.23576955307544],
     [-52.99193089007511, -21.23576953059324],
     [-48.23484617748847, -21.235769564160556],
     [-43.18015404170988, -21.235769294498468],
     [-43.180169190429005, 10.372173172469656],
     [-48.53216399204116, 10.372173325148935],
     [-57.74901562508815, 10.372173289588734],
     [-63.10073595010061, 10.372173304097

In [94]:
PamAmazon = ee.ImageCollection(amazonBiomas_reclass)

In [95]:
for el in PamAmazon.getInfo()['features']:
  print(el['bands'][0]['id'])

classification_1985
classification_1986
classification_1987
classification_1988
classification_1989
classification_1990
classification_1991
classification_1992
classification_1993
classification_1994
classification_1995
classification_1996
classification_1997
classification_1998
classification_1999
classification_2000
classification_2001
classification_2002
classification_2003
classification_2004
classification_2005
classification_2006
classification_2007
classification_2008
classification_2009
classification_2010
classification_2011
classification_2012
classification_2013
classification_2014
classification_2015
classification_2016
classification_2017
classification_2018
classification_2019
classification_2020


In [96]:
PamAmazon.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'classification_1985',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': 0,
      'max': 11},
     'dimensions': [141216, 117285],
     'crs': 'EPSG:4326',
     'crs_transform': [0.00026949458523585647,
      0,
      -81.23725625725228,
      0,
      -0.00026949458523585647,
      10.372038101972407]}],
   'version': 1632375433141026,
   'id': 'projects/mapbiomas-raisg/public/collection3/mapbiomas_raisg_panamazonia_collection3_integration_v2',
   'properties': {'system:footprint': {'type': 'LinearRing',
     'coordinates': [[-63.69537156255461, -21.235769568181407],
      [-60.127558032033875, -21.23576955307544],
      [-52.99193089007511, -21.23576953059324],
      [-48.23484617748847, -21.235769564160556],
      [-43.18015404170988, -21.235769294498468],
      [-43.180169190429005, 10.372173172469656],
      [-48.53216399204116, 10.372173325148935],
      [-57

In [97]:
# Região de Interesse a ROI
roi = ee.FeatureCollection('projects/ee-edivandocouto/assets/AmazonBasin_hybas_sa')

In [107]:
##Crie um mapa interativo 
Map = geemap.Map()
Map = geemap.Map(location=[-0.095659800460687,-51.31188826592861,], zoom_start= 5)
##Visualizando a imagem
vis_ = {"bands": ['classification_1985'], "min": 1, "max": 12}
vis_params = {"palette": ['129912', '32CD32', '687537', '70A5AE', 'BDB76B', 'FFFFB2', 'EA9999', '0000FF', 'ffffff', 'D5D5E5', 'AA0000'], "min": 0, "max": 11}
imagem = ee.Image(PamAmazon.select('classification_1985').first().clip(roi))
Map.addLayer(imagem, vis_params, 'classification_1985')
# ##Adicionando a feature  = Converte para Image (desenha o limite)
limite = ee.Image().paint(roi, 0, 2)
Map.addLayer(limite,{'palette':'black'}, 'ROI')
Map

Map(center=[-0.095659800460687, -51.31188826592861], controls=(WidgetControl(options=['position', 'transparent…

# Bacia Pam Amazonica para o mapbiomas coleção 3 

# import import south america borders
##Convertendo arquivo shp to json to feature
##Lendo o arquivo shp

In [119]:
# dados foram obtidos a partir da base 250k do IBGE https://geoftp.ibge.gov.br/cartas_e_mapas/bases_cartograficas_continuas/bc250/
# Baixei os dados e montei um arquivo no drive 
gdf = gpd.read_file('/content/drive/MyDrive/South_America/lml_pais_a.shp')
# meu interesse são
# brazil
# bolivia
# colombia
# guyana
# french_guyana
# peru
# suriname 
# venezuela 
brazil = gdf[gdf['nome']=='Brasil']
bolivia = gdf[gdf['nome']=='Bolivia']
colombia = gdf[gdf['nome']=='Colombia']
guyana = gdf[gdf['nome']=='Guyana']
french_guyane = gdf[gdf['nome']=='Guyane']
peru = gdf[gdf['nome']=='Perú']
suriname = gdf[gdf['nome']=='Suriname']
venezuela = gdf[gdf['nome']=='Venezuela']

In [120]:
brazil = brazil.to_json()
bolivia = bolivia.to_json()
colombia = colombia.to_json()
guyana = guyana.to_json()
french_guyane = french_guyane.to_json()
peru = peru.to_json()
suriname = suriname.to_json()
venezuela = venezuela.to_json()

In [ ]:
##Salvando o dado em shp e json
gdf.to_file("porto_alegre.json", driver='GeoJSON')
gdf.to_file("porto_alegre.shp")